# Vox2SDF
This is the demo of Vox2SDF.
Vox2SDF is a model that convert voxel object to DeepSDF representation.

## Clone things from git

In [1]:
!git clone --recursive https://github.com/TurtleJayWang/VoxelToSDF.git

Cloning into 'VoxelToSDF'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 216 (delta 26), reused 32 (delta 14), pack-reused 171 (from 1)
Receiving objects: 100% (216/216), 73.50 MiB | 42.17 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Submodule 'dataset' (https://github.com/TurtleJayWang/VoxelSDFDataset.git) registered for path 'dataset'
Cloning into '/kaggle/working/VoxelToSDF/dataset'...
remote: Enumerating objects: 95, done.        
remote: Counting objects: 100% (95/95), done.        
remote: Compressing objects: 100% (66/66), done.        
remote: Total 95 (delta 45), reused 71 (delta 24), pack-reused 0 (from 0)        
Receiving objects: 100% (95/95), 22.50 KiB | 7.50 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Submodule path 'dataset': checked out 'a0951ebbbad88fa5578a6a3dd2780623bdb5c7dd'
Submodule 'thirdparty/cuda_voxelizer' (https://github.com/Forceflow/cuda_

In [2]:
!cd VoxelToSDF; git pull

Already up to date.


## Download Data from Kaggle Dataset

In [3]:
import kagglehub
path = kagglehub.dataset_download("turtlejaywang/voxelsdfdataset")

# Configuration

In [4]:
import json
import os
config = {
    "train_epoch" : 2,
    "learning_rate" : 0.0005,
    "input_voxel_grid_size" : 64,
    "decoder_hidden_dimension" : 512,
    "decoder_num_hidden_layers" : 8,
    "latent_dimension" : 256,
    "num_sdf_samples" : 250000,
    "num_points_per_minor_batch" : 12500,
    "dataset_path" : os.path.join(path, "VoxelSDFDataset/data"),
    "cuda_voxelizer_path" : "thirdparty/cuda_voxelizer/cuda_voxelizer",
    "shapenet_pickle_name" : "shapenet_data.pkl",
    "shapenet_categories" : ["02747177"],
    "num_augment_data" : 4,
    "seperate_ratio" : [0.8, 0.2],
    "check_point_filename" : "checkpoints/checkpoint.pkl",
    "loss_filename" : "losses/loss.pkl",
    "is_train" : 1,
    "is_validation" : 1
}
with open("VoxelToSDF/Scripts/config.json", "w") as f:
    json.dump(config, f)

print("Complete configuration!")

Complete configuration!


# Train the model

In [5]:
!cd VoxelToSDF; python Scripts/train.py

/kaggle/working/VoxelToSDF/Scripts/train.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.network.load_state_dict(torch.load(f))
Checkpoint loaded checkpoints/chec

## Validation & Visualize Loss

In [6]:
!cd VoxelToSDF; python Scripts/validation.py

Traceback (most recent call last):
  File "/kaggle/working/VoxelToSDF/Scripts/validation.py", line 67, in <module>
    validation(network, validation_dataloader, cfg)
  File "/kaggle/working/VoxelToSDF/Scripts/validation.py", line 31, in validation
    for i, (voxel_tensor, point, sdf) in tqdm(enumerate(validation_loader, desc="Validation")):
TypeError: 'desc' is an invalid keyword argument for enumerate()


## Move checkpoint into a accessible folder

In [7]:
import os
import shutil
shutil.copytree('/kaggle/working/VoxelToSDF/checkpoints', '/kaggle/working/checkpoints')
shutil.copytree('/kaggle/working/VoxelToSDF/losses', '/kaggle/working/losses')

'/kaggle/working/losses'

# Visualize(To Do)